# Basic Usage

## Create a dataset

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset as TorchDataset


class Dataset(TorchDataset):
    def __init__(self, n_samples, x_size, seed=None):
        self.n_samples = n_samples
        self.x_size = x_size
        # Random generator
        self.rng = None
        if seed is not None:
            self.rng = torch.Generator().manual_seed(seed)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        x = torch.rand(self.x_size, generator=self.rng)
        y = x.mean()
        return x, y.unsqueeze(dim=0)


dataset = Dataset(n_samples=100, x_size=5, seed=100)

print(f"x: {dataset[0][0]}")
print(f"y: {dataset[0][1].item():.4f}")

## Create a model

In [ ]:
class Model(nn.Module):
    def __init__(self, x_size, hidden_size, seed=None):
        super().__init__()
        if seed is not None:
            torch.manual_seed(seed)
        self.lin1 = nn.Linear(x_size, hidden_size)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        return self.lin2(self.relu(self.lin1(x)))


model = Model(x_size=5, hidden_size=3, seed=100)

## Init IGC attribution method

In [ ]:
from igc import IntGradCorr

attr = IntGradCorr(model, dataset)

## Compute IGC attributions

In [ ]:
igc = attr.compute(           # Compute IGC attributions
    x_0=8,                    # with 8 random baselines sampled from the dataset
    y_idx=None,               # for all model output components
    n_steps=64,               # with 64 steps for each individual supporting IG
    batch_size=(4, 8, None),  # with 4 'x' samples, 8 baselines, and all y
    x_seed=100,               #   components per batch. It could also be defined
    x_0_seed=101,             #   by an integer as: batch_size=32
    n_x=None,                 # and 'x' sampled over the whole dataset
)

print()
print(igc)